In [1]:
import os
from shutil import copy

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model, Model

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

from pprint import pprint

C:\Users\Sophie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
def copyImage(src, label, tr_te, dst=""):
        cat = os.path.join(tr_te, label)
        new_dst = os.path.join(dst,cat)
        
        #print(src)
        #print(cat)
        #print(new_dst)
        
        # check if directory exists
        if os.path.isdir(new_dst):
            pass
        else:        
            # make diretory
            os.mkdir(new_dst)

        #print(new_dst)
        copy(src, new_dst)

In [28]:
#generateData("./CUB_200_2011/CUB_200_2011/CUB_200_2011/train.txt", "./Outex_TC_00030.tar/Outex-TC-00030/images/", "C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2/Outex-TC-00030/test/")

drt = "./CUB_200_2011/CUB_200_2011/CUB_200_2011/"
drt_images = "./CUB_200_2011/CUB_200_2011/CUB_200_2011/images"
dst = "./CUB_200_2011_generator/"

# images.txt
with open(os.path.join(drt, "images.txt"), "r", encoding='utf-8') as f:
    images_txt = f.read().splitlines()

# train_test_split.txt
with open(os.path.join(drt, "train_test_split.txt"), "r", encoding='utf-8') as f:
    tr_te_txt = f.read().splitlines()

# image_class_labels.txt
with open(os.path.join(drt, "image_class_labels.txt"), "r", encoding='utf-8') as f:
    class_labels_txt = f.read().splitlines()

print(images_txt[:2])
print(tr_te_txt[:2])
print(class_labels_txt[:2])
print("# images_txt: ", len(images_txt))
print("# tr_te_txt: ", len(tr_te_txt))
print("# class_labels_txt: ", len(class_labels_txt))

#birdspecies_subset = {"cardinal": 17, "american_goldﬁnch": 47, "evening_grosbeak": 55, "song_sparrow": 129, "tree_sparrow": 130, "house_sparrow": 118, "fox_sparrow":120, "canada_warbler":162, "myrtle warbler": 171, "house_wren": 196 , "red-winged_blackbird": 10, "boat_tailed_grackle":49}
birdspecies_subset = {"17": "cardinal", "47": "american_goldﬁnch", "55": "evening_grosbeak", "129": "song_sparrow", "130": "tree_sparrow", "118": "house_sparrow", "120": "fox_sparrow", "162": "canada_warbler", "171": "myrtle warbler", "196": "house_wren", "10": "red-winged_blackbird", "49": "boat_tailed_grackle"}
label_subset = set(birdspecies_subset.keys())
print("label_subset: ", label_subset)

for i, label in enumerate(class_labels_txt):
    label = label.split(' ')[-1]
    if label in label_subset:
        file = images_txt[i].split(' ')[-1]
        tr_te = tr_te_txt[i].split(' ')[-1]
        if tr_te == "1":
            tr_te = "train"
        elif tr_te == "0":
            tr_te = "test"
        else:
            break
        
        print(file)
        src = os.path.join(drt_images, file)
        copyImage(src, label, tr_te, dst)
        #break
        

['1 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg', '2 001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg']
['1 0', '2 1']
['1 1', '2 1']
# images_txt:  11788
# tr_te_txt:  11788
# class_labels_txt:  11788
label_subset:  {'129', '162', '17', '196', '49', '118', '10', '130', '47', '55', '120', '171'}
010.Red_winged_Blackbird/Red_Winged_Blackbird_0091_4096.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0059_4612.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0054_4625.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0022_4483.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0017_4116.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0079_4527.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0075_4953.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0005_5636.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0058_4141.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0074_4146.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0042_3635.jpg
010.Red_wing

047.American_Goldfinch/American_Goldfinch_0107_32618.jpg
047.American_Goldfinch/American_Goldfinch_0022_32111.jpg
047.American_Goldfinch/American_Goldfinch_0129_32625.jpg
047.American_Goldfinch/American_Goldfinch_0064_32142.jpg
047.American_Goldfinch/American_Goldfinch_0089_32152.jpg
047.American_Goldfinch/American_Goldfinch_0014_32154.jpg
047.American_Goldfinch/American_Goldfinch_0056_32169.jpg
047.American_Goldfinch/American_Goldfinch_0106_32182.jpg
047.American_Goldfinch/American_Goldfinch_0100_32183.jpg
047.American_Goldfinch/American_Goldfinch_0122_32186.jpg
047.American_Goldfinch/American_Goldfinch_0118_32210.jpg
047.American_Goldfinch/American_Goldfinch_0121_32725.jpg
047.American_Goldfinch/American_Goldfinch_0029_32218.jpg
047.American_Goldfinch/American_Goldfinch_0026_32222.jpg
047.American_Goldfinch/American_Goldfinch_0103_32225.jpg
047.American_Goldfinch/American_Goldfinch_0003_32236.jpg
047.American_Goldfinch/American_Goldfinch_0105_32238.jpg
049.Boat_tailed_Grackle/Boat_Ta

118.House_Sparrow/House_Sparrow_0137_111219.jpg
118.House_Sparrow/House_Sparrow_0052_112252.jpg
118.House_Sparrow/House_Sparrow_0094_112773.jpg
118.House_Sparrow/House_Sparrow_0079_113288.jpg
118.House_Sparrow/House_Sparrow_0119_111471.jpg
118.House_Sparrow/House_Sparrow_0084_111300.jpg
118.House_Sparrow/House_Sparrow_0046_112845.jpg
118.House_Sparrow/House_Sparrow_0036_112847.jpg
118.House_Sparrow/House_Sparrow_0133_112368.jpg
118.House_Sparrow/House_Sparrow_0080_111099.jpg
118.House_Sparrow/House_Sparrow_0002_112905.jpg
118.House_Sparrow/House_Sparrow_0067_112913.jpg
118.House_Sparrow/House_Sparrow_0086_111385.jpg
118.House_Sparrow/House_Sparrow_0030_111387.jpg
118.House_Sparrow/House_Sparrow_0053_111388.jpg
118.House_Sparrow/House_Sparrow_0055_111393.jpg
118.House_Sparrow/House_Sparrow_0143_111400.jpg
118.House_Sparrow/House_Sparrow_0145_112703.jpg
118.House_Sparrow/House_Sparrow_0014_112947.jpg
118.House_Sparrow/House_Sparrow_0015_111085.jpg
118.House_Sparrow/House_Sparrow_0092_111

130.Tree_Sparrow/Tree_Sparrow_0018_123574.jpg
130.Tree_Sparrow/Tree_Sparrow_0095_124090.jpg
130.Tree_Sparrow/Tree_Sparrow_0125_123078.jpg
130.Tree_Sparrow/Tree_Sparrow_0058_123591.jpg
130.Tree_Sparrow/Tree_Sparrow_0101_124104.jpg
130.Tree_Sparrow/Tree_Sparrow_0119_124114.jpg
130.Tree_Sparrow/Tree_Sparrow_0080_124120.jpg
130.Tree_Sparrow/Tree_Sparrow_0113_123613.jpg
130.Tree_Sparrow/Tree_Sparrow_0039_124140.jpg
130.Tree_Sparrow/Tree_Sparrow_0057_123665.jpg
130.Tree_Sparrow/Tree_Sparrow_0094_124974.jpg
130.Tree_Sparrow/Tree_Sparrow_0068_125230.jpg
130.Tree_Sparrow/Tree_Sparrow_0025_124233.jpg
130.Tree_Sparrow/Tree_Sparrow_0105_123227.jpg
130.Tree_Sparrow/Tree_Sparrow_0060_123743.jpg
130.Tree_Sparrow/Tree_Sparrow_0086_123751.jpg
130.Tree_Sparrow/Tree_Sparrow_0088_125305.jpg
130.Tree_Sparrow/Tree_Sparrow_0090_110145.jpg
130.Tree_Sparrow/Tree_Sparrow_0121_124296.jpg
130.Tree_Sparrow/Tree_Sparrow_0127_125322.jpg
130.Tree_Sparrow/Tree_Sparrow_0123_125324.jpg
130.Tree_Sparrow/Tree_Sparrow_0036

196.House_Wren/House_Wren_0139_187177.jpg
196.House_Wren/House_Wren_0137_187273.jpg
196.House_Wren/House_Wren_0023_187192.jpg
196.House_Wren/House_Wren_0062_187194.jpg
196.House_Wren/House_Wren_0035_187708.jpg
196.House_Wren/House_Wren_0085_187723.jpg
196.House_Wren/House_Wren_0043_187224.jpg
196.House_Wren/House_Wren_0094_187226.jpg
196.House_Wren/House_Wren_0107_187230.jpg
196.House_Wren/House_Wren_0099_187240.jpg
196.House_Wren/House_Wren_0088_187243.jpg
196.House_Wren/House_Wren_0097_187760.jpg
196.House_Wren/House_Wren_0090_187762.jpg
196.House_Wren/House_Wren_0125_187251.jpg
196.House_Wren/House_Wren_0086_187815.jpg
196.House_Wren/House_Wren_0127_187832.jpg
196.House_Wren/House_Wren_0017_187330.jpg
196.House_Wren/House_Wren_0122_187331.jpg
196.House_Wren/House_Wren_0022_187848.jpg
196.House_Wren/House_Wren_0098_187368.jpg
196.House_Wren/House_Wren_0045_187374.jpg
196.House_Wren/House_Wren_0129_187376.jpg
196.House_Wren/House_Wren_0118_187383.jpg
196.House_Wren/House_Wren_0072_187

In [ ]:
#image size of 32×32

datagen = ImageDataGenerator()

train_batchsize = 32 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"./Outex-TC-00030/train/",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"./Outex-TC-00030/test/",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])